In [182]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adadelta, Adam
from keras.preprocessing.image import ImageDataGenerator
from typing import Tuple, List, Dict
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [160]:
label_map={"0":"ANGRY","1":"HAPPY","2":"SAD","3":"SURPRISE","4":"NEUTRAL"}
data = pd.read_csv("fer2013.csv")
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


The data is divided into three subsets: Training, PublicTest (validation)& PrivateTest

In [161]:
def parse_df(source_df):
    X = [np.fromstring(i, dtype=np.int16, sep=' ') for i in source_df["pixels"]]
    y = source_df.values[:, :1]
    y = y.reshape(-1,)
    y = y.astype("int")
    return X, y

train_df = data[data["Usage"]=="Training"]
val_df = data[data["Usage"]=="PublicTest"]
test_df = data[data["Usage"]=="PrivateTest"]

features_train, targets_train = parse_df(train_df)
features_val, targets_val = parse_df(val_df)
features_test, targets_test = parse_df(test_df)

print(np.asarray(features_train).shape)
print(np.asarray(features_val).shape)
print(np.asanyarray(features_test).shape)
targets_train.reshape(-1,)

(28709, 2304)
(3589, 2304)
(3589, 2304)


array([0, 0, 2, ..., 4, 0, 4])

In [163]:
INPUT_SHAPE = (48, 48)
INPUT_DIM = INPUT_SHAPE[0]

In [164]:
def reconstruct_dataset(X_vec):
    y_vec = [[index]*len(value) for index, value in enumerate(X_vec)]

    X = list(itertools.chain(*X_vec))  # concatenate all Xs
    y = list(itertools.chain(*y_vec))  # conacatenate all ys

    return X, y

In [165]:
X_train_data = np.array(X_train_fin).reshape(-1, INPUT_DIM, INPUT_DIM, 1)
X_train_data = X_train_data/255.
y_train_data = np.array(y_train_fin)
y_train_data = to_categorical(y_train_data)

X_val_data = np.array(X_val_fin).reshape(-1, INPUT_DIM, INPUT_DIM, 1)
X_val_data = X_val_data/255.
y_val_data = np.array(y_val_fin)
y_val_data = to_categorical(y_val_data)

X_test_data = np.array(X_test).reshape(-1, INPUT_DIM, INPUT_DIM, 1)
X_test_data = X_test_data/255.
y_test_data = np.array(y_test)
y_test_data = to_categorical(y_test_data)


In [166]:
def reshape_img_input(x: np.ndarray,
                      img_rows: int = 28,
                      img_cols: int = 28,
                      normalize: bool = True) -> Tuple:
  
    
    print(f'Original shape of nd-array: {x.shape}')
    
    if K.image_data_format() == 'channels_first':
        
        # the channel dimension goes to the front
        x1 = x.reshape(x.shape[0], 1, img_rows, img_cols)

        input_shape = (1, img_rows, img_cols)

    else:
        x1 = x.reshape(x.shape[0], img_rows, img_cols, 1)

        input_shape = (img_rows, img_cols, 1)

    x1 = x1.astype('float32')

    if normalize:
        x1 /= 255.
    
    
    return x1, input_shape

In [120]:
X_test, input_shape = reshape_img_input(x=X_test,
                                         img_rows=48, 
                                         img_cols=48,
                                         normalize=True)

Original shape of nd-array: (1800, 2304)


In [126]:
X_test_data = X_test.reshape(X_test.shape[0], 1, 48, 48).astype('float32')

In [212]:
#X_train_data = X_train_data.str.extract(r'(\d+)')

In [215]:
X_train_data, input_shape = reshape_img_input(x=X_train_data,
                                         img_rows=48, 
                                         img_cols=48,
                                         normalize=True)

Original shape of nd-array: (30856, 48, 48, 1)


In [ ]:
X_train_data = X_train.reshape(X_train_data.shape[0], 1, 48, 48).astype('float32')

# Building the Model

In [202]:
#hyperparameters
batch_size = 1024

epochs = 100

In [266]:
model = Sequential()  
model.add(Conv2D(48, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:]), padding='same'))  
model.add(Conv2D(48,kernel_size= (3, 3), activation='relu', padding='same'))    
model.add(MaxPooling2D(pool_size=(1,1)))  
model.add(Dropout(0.3))  
#2nd convolution layer  
model.add(Conv2D(48, (3, 3), activation='relu', padding='same'))  
model.add(Conv2D(48, (3, 3), activation='relu', padding='same'))    
model.add(MaxPooling2D(pool_size=(1,1)))  
model.add(Dropout(0.3))  
#3rd convolution layer  
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))  
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))    
model.add(MaxPooling2D(pool_size=(1,1)))  
model.add(Flatten())  
model.add(Dense(6,activation='softmax'))  
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_101 (Conv2D)         (None, 1, 48, 48)         20784     
                                                                 
 conv2d_102 (Conv2D)         (None, 1, 48, 48)         20784     
                                                                 
 max_pooling2d_44 (MaxPoolin  (None, 1, 48, 48)        0         
 g2D)                                                            
                                                                 
 dropout_26 (Dropout)        (None, 1, 48, 48)         0         
                                                                 
 conv2d_103 (Conv2D)         (None, 1, 48, 48)         20784     
                                                                 
 conv2d_104 (Conv2D)         (None, 1, 48, 48)         20784     
                                                     

In [267]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [268]:
model.fit(
    X_train_data,
    y_train_data,
    batch_size = batch_size,
    epochs = epochs,
    verbose = 1,
    validation_split = 0.1,)

Epoch 1/100


ValueError: in user code:

    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of la

# Building the second model

In [265]:
model2 = Sequential()  
model2.add(Conv2D(48, kernel_size=(4, 4), activation='relu', input_shape=(X_train.shape[1:]), padding='same'))  
model2.add(Conv2D(48,kernel_size= (4, 4), activation='relu', padding='same'))    
model2.add(MaxPooling2D(pool_size=(1,1)))  
model2.add(Dropout(0.7))  
#2nd convolution layer  
model2.add(Conv2D(64, (4, 4), activation='relu', padding='same'))  
model2.add(Conv2D(64, (4, 4), activation='relu', padding='same'))    
model2.add(MaxPooling2D(pool_size=(1,1)))  
model2.add(Dropout(0.1))  
#3rd convolution layer  
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))  
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))    
model2.add(MaxPooling2D(pool_size=(1,1)))  
model2.add(Flatten())  
model2.add(Dense(6,activation='softmax'))  
model2.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_95 (Conv2D)          (None, 1, 48, 48)         36912     
                                                                 
 conv2d_96 (Conv2D)          (None, 1, 48, 48)         36912     
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 1, 48, 48)        0         
 g2D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 1, 48, 48)         0         
                                                                 
 conv2d_97 (Conv2D)          (None, 1, 48, 64)         49216     
                                                                 
 conv2d_98 (Conv2D)          (None, 1, 48, 64)         65600     
                                                     

In [263]:
model2.compile(optimizer = 'adam', loss = 'crossentropy', metrics = ['accuracy'])

In [264]:
model2.fit(X_train_data, y_train_data, epochs = 25)

Epoch 1/25


ValueError: in user code:

    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of la

# Evaluation

In [ ]:
score = model.evaluate(x_test,
                       y_test,
                       verbose=1)

print(f'Test loss: {score[0]}:')
print('Test accuracy: {:.3f} %'.format(100 * score[1]))

In [ ]:
y_test_probs = model.predict(x_test) 
y_test_pred=np.argmax(y_test_probs, axis=1)
y_test_pred